In [42]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.ensemble import RandomForestClassifier

In [31]:
df = pd.read_csv("data/reviews_matrix.csv.gz")
df['rating'] = df.rating.apply(lambda x: 0 if x <= 6 else 1)

In [32]:
X=df.iloc[:,:-1]
X

,feel,medication,anxiety,doctor,medicine,weight,better,sleep,work,really,...,taken,able,experience,finally,blood,stop,gone,recommend,worse,daily
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91842,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91843,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91844,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91845,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0


In [33]:
y=df.rating
y

0        1
1        1
2        0
3        1
4        1
        ..
91842    1
91843    1
91844    1
91845    1
91846    1
Name: rating, Length: 91847, dtype: int64

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20,random_state=100)

In [35]:
## Lin reg

In [36]:
std_scaler = StandardScaler().fit(X_train) 
X_train_scaled = std_scaler.transform(X_train)

In [37]:
X_test_scaled=std_scaler.transform(X_test)

In [38]:
X_train_const_scaled = sm.add_constant(X_train_scaled)

model = sm.OLS(y_train, X_train_const_scaled).fit()
predictions_train = model.predict(X_train_const_scaled) 

X_test_const_scaled = sm.add_constant(X_test_scaled)
predictions_test = model.predict(X_test_const_scaled) 
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     116.1
Date:                Thu, 10 Mar 2022   Prob (F-statistic):               0.00
Time:                        10:25:53   Log-Likelihood:                -42251.
No. Observations:               73477   AIC:                         8.462e+04
Df Residuals:                   73418   BIC:                         8.516e+04
Df Model:                          58                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7194      0.002    453.262      0.0

In [39]:
## Classification?

In [40]:
y.value_counts()

1    66068
0    25779
Name: rating, dtype: int64

In [47]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [48]:
y_train_pred_rf = rf.predict(X_train)
y_test_pred_rf  = rf.predict(X_test)

In [49]:
def model_performance_class(y_train, y_pred_train, y_test, y_pred_test):

    from sklearn.metrics import cohen_kappa_score, classification_report 


    print("Results obtained for the TRAIN SET")
    print("==================================")
    print("The Cohen's Kappa is: {:.2f}".format(cohen_kappa_score(y_train, y_pred_train)))
    print(classification_report(y_train, y_pred_train))
    print("==================================")
    print("Results obtained for the TEST SET")
    print("The Cohen's Kappa is: {:.2f}".format(cohen_kappa_score(y_test, y_pred_test)))
    print(classification_report(y_test, y_pred_test))

In [50]:
model_performance_class(y_train, y_train_pred_rf, y_test, y_test_pred_rf)

Results obtained for the TRAIN SET
The Cohen's Kappa is: 0.82
              precision    recall  f1-score   support

           0       0.94      0.80      0.87     20621
           1       0.93      0.98      0.95     52856

    accuracy                           0.93     73477
   macro avg       0.94      0.89      0.91     73477
weighted avg       0.93      0.93      0.93     73477

Results obtained for the TEST SET
The Cohen's Kappa is: 0.16
              precision    recall  f1-score   support

           0       0.48      0.23      0.31      5158
           1       0.75      0.90      0.82     13212

    accuracy                           0.71     18370
   macro avg       0.61      0.57      0.56     18370
weighted avg       0.67      0.71      0.68     18370



In [51]:
category_0 = df[df['rating'] == 0]
category_1 = df[df['rating'] == 1]
category_1
category_0

,feel,medication,anxiety,doctor,medicine,weight,better,sleep,work,really,...,able,experience,finally,blood,stop,gone,recommend,worse,daily,rating
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
5,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91821,0,0,0,0,0,0,0,0,0,0,...,0,0,0,3,0,0,0,0,0,0
91825,3,2,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
91827,0,0,0,1,1,0,0,3,0,0,...,0,0,1,0,0,0,0,0,0,0
91829,0,0,0,0,0,1,0,0,0,2,...,0,0,0,0,1,0,0,0,0,0


In [52]:
category_1 = category_1.sample(len(category_0))
print(category_0.shape)
print(category_1.shape)

(25779, 60)
(25779, 60)


In [53]:
df_down = pd.concat([category_0, category_1], axis=0)
df_down = df_down.sample(frac=1)
df_down['rating'].value_counts()
df_down.reset_index(drop=True, inplace=True)
df_down

,feel,medication,anxiety,doctor,medicine,weight,better,sleep,work,really,...,able,experience,finally,blood,stop,gone,recommend,worse,daily,rating
0,1,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51553,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
51554,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
51555,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
51556,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [55]:
df_down['rating'].value_counts()

1    25779
0    25779
Name: rating, dtype: int64

In [56]:
X_down=df_down.iloc[:,:-1]
y_down=df_down['rating']

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X_down, y_down, test_size=.20,random_state=100)

In [62]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [63]:
y_train_pred_rf = rf.predict(X_train)
y_test_pred_rf  = rf.predict(X_test)

In [64]:
model_performance_class(y_train, y_train_pred_rf, y_test, y_test_pred_rf)

Results obtained for the TRAIN SET
The Cohen's Kappa is: 0.83
              precision    recall  f1-score   support

           0       0.90      0.94      0.92     20611
           1       0.94      0.89      0.91     20635

    accuracy                           0.92     41246
   macro avg       0.92      0.92      0.92     41246
weighted avg       0.92      0.92      0.92     41246

Results obtained for the TEST SET
The Cohen's Kappa is: 0.24
              precision    recall  f1-score   support

           0       0.62      0.62      0.62      5168
           1       0.62      0.62      0.62      5144

    accuracy                           0.62     10312
   macro avg       0.62      0.62      0.62     10312
weighted avg       0.62      0.62      0.62     10312

